## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Baruun-Urt,MN,46.6806,113.2792,60.55,75,100,13.73,overcast clouds
1,1,Bluff,NZ,-46.6000,168.3333,49.80,89,97,4.85,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,37.51,87,0,18.41,clear sky
3,3,Tura,IN,25.5198,90.2201,79.95,83,100,5.48,moderate rain
4,4,Fort Saint James,CA,54.4333,-124.2500,60.04,70,67,2.15,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Tura,IN,25.5198,90.2201,79.95,83,100,5.48,moderate rain
10,10,Ahuimanu,US,21.4447,-157.8378,83.91,58,40,11.50,scattered clouds
15,15,Butaritari,KI,3.0707,172.7902,82.85,74,22,8.32,few clouds
16,16,Marovoay,MG,-16.1000,46.6333,73.54,83,100,2.84,overcast clouds
17,17,Tabuk,PH,17.4189,121.4443,87.64,60,20,2.77,few clouds
20,20,Jamestown,US,42.0970,-79.2353,72.99,78,90,3.78,overcast clouds
22,22,Makakilo City,US,21.3469,-158.0858,82.72,54,1,13.80,clear sky
24,24,San Juan,AR,-31.5375,-68.5364,72.73,41,13,10.13,few clouds
28,28,Vaini,TO,-21.2000,-175.2000,80.76,74,20,9.22,few clouds
34,34,San Patricio,US,28.0170,-97.5169,87.55,72,5,8.79,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             337
City                   337
Country                337
Lat                    337
Lng                    337
Max Temp               337
Humidity               337
Cloudiness             337
Wind Speed             337
Current Description    337
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis = 0)
clean_df.count()

Unnamed: 0             337
City                   337
Country                337
Lat                    337
Lng                    337
Max Temp               337
Humidity               337
Cloudiness             337
Wind Speed             337
Current Description    337
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tura,IN,79.95,moderate rain,25.5198,90.2201,
10,Ahuimanu,US,83.91,scattered clouds,21.4447,-157.8378,
15,Butaritari,KI,82.85,few clouds,3.0707,172.7902,
16,Marovoay,MG,73.54,overcast clouds,-16.1000,46.6333,
17,Tabuk,PH,87.64,few clouds,17.4189,121.4443,
20,Jamestown,US,72.99,overcast clouds,42.0970,-79.2353,
22,Makakilo City,US,82.72,clear sky,21.3469,-158.0858,
24,San Juan,AR,72.73,few clouds,-31.5375,-68.5364,
28,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
34,San Patricio,US,87.55,clear sky,28.0170,-97.5169,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url  = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
string_to_NaN_df = hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
string_to_NaN_df
clean_hotel_df = hotel_df.dropna(axis = 0)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tura,IN,79.95,moderate rain,25.5198,90.2201,Hotel RIKMAN Continental
10,Ahuimanu,US,83.91,scattered clouds,21.4447,-157.8378,Paradise Bay Resort
15,Butaritari,KI,82.85,few clouds,3.0707,172.7902,Isles Sunset Lodge
17,Tabuk,PH,87.64,few clouds,17.4189,121.4443,Golden Berries Hotel and Convention
20,Jamestown,US,72.99,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [24]:
# 8a. Create the output File (CSV)
import os.path
output_data_file = os.path.join("../Vacation_Search", "WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))